# Train Models - Step 3 & 4

In [59]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
from itertools import product

In [2]:
# load environment variables from .env file
load_dotenv()  
openai.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

## Defaults - Step 3

In [ ]:
# upload training file
training_file = client.files.create(
    file=open("./tuning_files/train.jsonl", "rb"),
    purpose="fine-tune"
)
# upload validation file
valid_file = client.files.create(
    file=open("./tuning_files/valid.jsonl", "rb"),
    purpose="fine-tune"
)

In [ ]:
# create a training job
# model selected based on OpenAI documentation recommendations as of 15-07-2024
default_train = client.fine_tuning.jobs.create(
  training_file= training_file.id, 
  validation_file=valid_file.id,
  model="gpt-3.5-turbo-0125",
  suffix="default-aci-bench",
  seed = 42
)

In [32]:
default_train = client.fine_tuning.jobs.retrieve("ftjob-EytdC3lvCLuQ70T3OCQb3Rf4")

In [39]:
default_train.status

'succeeded'

## Manual Testing - Step 4

In [73]:
# define lists of all hyperparameters 
learning_rates = [0.5, 1, 2]
epochs = [3, 10]
batches = [1, 32, 67]

# make dictionary of hyperparameters

def make_dict(learning_rates, epochs, batches):
    hyperparameters_dict = {}
    count = 0
    all_combinations = product(learning_rates, epochs, batches)
    for rate, epoch, batch in all_combinations:
        print(count, rate, epoch, batch)
        hyperparameters_dict[f"{count}"] = {
                        "learning_rate_multiplier": str(rate),
                        "n_epochs": str(epoch),
                        "batch_size": str(batch)
                    }
        count += 1
    return hyperparameters_dict

hyperparameters_dict = make_dict(learning_rates, epochs, batches)   
print(hyperparameters_dict)

0 0.5 3 1
1 0.5 3 32
2 0.5 3 67
3 0.5 10 1
4 0.5 10 32
5 0.5 10 67
6 1 3 1
7 1 3 32
8 1 3 67
9 1 10 1
10 1 10 32
11 1 10 67
12 2 3 1
13 2 3 32
14 2 3 67
15 2 10 1
16 2 10 32
17 2 10 67
{'0': {'learning_rate_multiplier': '0.5', 'n_epochs': '3', 'batch_size': '1'}, '1': {'learning_rate_multiplier': '0.5', 'n_epochs': '3', 'batch_size': '32'}, '2': {'learning_rate_multiplier': '0.5', 'n_epochs': '3', 'batch_size': '67'}, '3': {'learning_rate_multiplier': '0.5', 'n_epochs': '10', 'batch_size': '1'}, '4': {'learning_rate_multiplier': '0.5', 'n_epochs': '10', 'batch_size': '32'}, '5': {'learning_rate_multiplier': '0.5', 'n_epochs': '10', 'batch_size': '67'}, '6': {'learning_rate_multiplier': '1', 'n_epochs': '3', 'batch_size': '1'}, '7': {'learning_rate_multiplier': '1', 'n_epochs': '3', 'batch_size': '32'}, '8': {'learning_rate_multiplier': '1', 'n_epochs': '3', 'batch_size': '67'}, '9': {'learning_rate_multiplier': '1', 'n_epochs': '10', 'batch_size': '1'}, '10': {'learning_rate_multiplier

In [90]:
# count is adjusted as each hyperparameter combination is tried (count in range of 0 to 17)
count = 17
# create training job
temp_suffix = f"id{count}-{hyperparameters_dict[f"{count}"]["learning_rate_multiplier"]}-{hyperparameters_dict[f"{count}"]["n_epochs"]}-{hyperparameters_dict[f"{count}"]["batch_size"]}"
client.fine_tuning.jobs.create(
                    training_file= training_file, 
                    validation_file=valid_file,
                    model="gpt-3.5-turbo-0125",
                    suffix=temp_suffix,
                    seed = 42, 
                    hyperparameters=hyperparameters_dict[f"{count}"]
                    )

FineTuningJob(id='ftjob-ZMBUXYrPAI2CPXMrCmbKZBjJ', created_at=1721065489, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=10, batch_size=67, learning_rate_multiplier=2.0), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-PM149zGSOC87X64hmzqjDsm3', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-IUzCMcsHvKtgOUAZ7S0BTflQ', validation_file='file-i9Cn9gdkVGmZuE9qUIJOYcfy', estimated_finish=None, integrations=[], user_provided_suffix='id17-2-10-67')